In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gpflow as gpf
import tensorflow as tf

from gpflow.utilities import print_summary
from gpflow.ci_utils import ci_niter

from reggae.data_loaders import load_barenco_puma, DataHolder
from reggae.gp import LinearResponseKernel, LinearResponseMeanFunction
from reggae.gp.sim_kernels import LinearCoregionalization

gpf.config.set_default_float(np.float64)
gpf.config.set_default_summary_fmt("notebook")
np.random.seed(0)
%matplotlib inline

f64 = np.float64
MAXITER = ci_niter(2000)
plt.style.use('ggplot')


In [ ]:
m_observed, f_observed, σ2_m_pre, σ2_f_pre, t = load_barenco_puma()
m_df, m_observed = m_observed 
f_df, f_observed = f_observed
# Shape of m_observed = (replicates, genes, times)
m_observed = m_observed
f_observed = f_observed
data = (m_observed, f_observed)

σ2_m_pre = f64(σ2_m_pre)
σ2_f_pre = f64(σ2_f_pre)
noise_data = (σ2_m_pre, σ2_f_pre)


num_genes = m_observed.shape[1]
N_m = m_observed.shape[2]
granularity = 100
τ = np.linspace(0, 12, granularity)
time = (t, τ, None)
data = DataHolder(data, noise_data, time)

Y_var = data.σ2_m_pre[0].reshape(-1)

m_obs = m_observed #(Nr, J, Nm)
m_obs[0, 3] = np.array([2, 2.5, 1.5, 1, 0.5, 0.6, 0.3])
N = 35#m_obs.shape[2]  # number of points
D = 1  # number of input dimensions
M = N  # number of inducing points
L = 2  # number of latent GPs
P = m_obs.shape[1]  # number of observations = output dimensions
print(N)

In [ ]:
print(m_obs.shape)
plt.plot(m_obs[0, 0])
plt.plot([2, 2.5, 1.5, 1, 0.5, 0.6, 0.3])

In [ ]:
def generate_data(N=100):
    X = np.random.rand(N)[:, None] * 10 - 5  # Inputs = N x D
    G = np.hstack((0.5 * np.sin(3 * X) + X, 3.0 * np.cos(X) - X))  # G = N x L
    W = np.array([[0.5, -0.3, 1.5], [-0.4, 0.43, 0.0]])  # L x P
    F = np.matmul(G, W)  # N x P
    Y = F + np.random.randn(*F.shape) * [0.2, 0.2, 0.2]

    return X, Y

In [ ]:
X = np.c_[[np.float64(t) for _ in range(num_genes)]].reshape(-1, 1)
X, Y = XY = X, np.tile(m_obs[0].reshape(-1, 1), (1, 5))#m_obs[0].T
Zinit = X#t[:, None] #np.linspace(-5, 5, M)[:, None]
print(X.shape, Y.shape)

In [ ]:
def plot_model(m, lower=0, upper=14.0):
    pX = np.linspace(lower, upper, 100)[:, None]
    pY, pYv = m.predict_y(pX)
    if pY.ndim == 3:
        pY = pY[:, 0, :]
    print(X.shape, Y.shape)
    for i in range(Y.shape[1]):        
        plt.plot(X[:,0], Y[:,i], "x", label=i)
    plt.gca().set_prop_cycle(None)
    plt.plot(pX, pY)
    for i in range(pY.shape[1]):
        top = pY[:, i] + 2.0 * pYv[:, i] ** 0.5
        bot = pY[:, i] - 2.0 * pYv[:, i] ** 0.5
        plt.fill_between(pX[:, 0], top, bot, alpha=0.3)
    plt.xlabel("X")
    plt.ylabel("f")
    plt.title(f"ELBO: {m.elbo(XY):.3}")
    plt.plot(Z, Z * 0.0, "o")
    plt.legend()

In [ ]:
def optimize_model_with_scipy(model):
    optimizer = gpf.optimizers.Scipy()
    log = optimizer.minimize(
        model.training_loss_closure(XY),
        variables=model.trainable_variables,
        method="l-bfgs-b",
        options={"disp": True, "maxiter": MAXITER},
    )
    return log

# optimize_model_with_scipy(m)


In [ ]:
# Create list of kernels for each output
kern_list = [gpf.kernels.SquaredExponential() + gpf.kernels.Linear() for _ in range(L)]
kern_list = [LinearResponseKernel(data, Y_var) + gpf.kernels.Linear() for _ in range(L)]

# Create multi-output kernel from kernel list
kernel = gpf.kernels.LinearCoregionalization(
    kern_list, W=np.random.randn(P, L)
)  # Notice that we initialise the mixing matrix W
# initialisation of inducing input locations (M random points from the training inputs)
Z = Zinit.copy()
# create multi-output inducing variables from Z
iv = gpf.inducing_variables.SharedIndependentInducingVariables(
    gpf.inducing_variables.InducingPoints(Z)
)


In [ ]:
# initialize mean of variational posterior to be of shape MxL
q_mu = np.zeros((M, L))
# initialize \sqrt(Σ) of variational posterior to be of shape LxMxM
q_sqrt = np.repeat(np.eye(M)[None, ...], L, axis=0) * 1.0

# create SVGP model as usual and optimize
m = gpf.models.SVGP(
    kernel, gpf.likelihoods.Gaussian(), inducing_variable=iv, q_mu=q_mu, q_sqrt=q_sqrt
)
# m = gpf.models.VGP(XY, kernel, gpf.likelihoods.Gaussian())



In [ ]:
optimize_model_with_scipy(m)

In [ ]:
# create multi-output kernel
kernel = gpf.kernels.SharedIndependent(
    LinearResponseKernel(data, Y_var), output_dim=P
)
# initialization of inducing input locations (M random points from the training inputs)
Z = Zinit.copy()
# create multi-output inducing variables from Z
iv = gpf.inducing_variables.SharedIndependentInducingVariables(
    gpf.inducing_variables.InducingPoints(Z)
)


In [ ]:
# create SVGP model as usual and optimize
# m = gpf.models.SVGP(kernel, gpf.likelihoods.Gaussian(), inducing_variable=iv, num_latent_gps=P)
m = gpf.models.VGP(XY, kernel, gpf.likelihoods.Gaussian(), num_latent_gps=P)
print_summary(m)


In [ ]:
print(LinearResponseKernel(data, Y_var).K(X).shape)

In [ ]:
w = tf.random.normal((5, 2), dtype='float64')
K = tf.stack([k.K_diag(X) for k in kernel.kernels], axis=1)
# k = tf.random.normal((2, 12, 12))
print(w[:,:,None,None].shape)


In [ ]:
from reggae.utilities import broadcast_tile
print(K.shape)
print(K[:,:,None, None].shape)
w = broadcast_tile(w, 7, 1)
wt = tf.transpose(w)
print(wt.shape)
print(tf.reduce_sum(K[:,:,None, None] * wt[None, :,:,None]* wt[None, :, None, :], axis=1).shape)

In [ ]:
K = m.kernel(X) + tf.eye(35, dtype='float64') * 1e-6
L = tf.linalg.cholesky(K)


In [ ]:
L_tiled = tf.tile(tf.expand_dims(L, 0), tf.stack([2, 1, 1]))

In [ ]:
print_summary(m)


In [ ]:
plot_model(m)


In [ ]:
plot_model(m)


In [ ]:
print_summary(m)

In [ ]:
m.kernel.W.numpy()

In [ ]:
plt.imshow(m.kernel.W.numpy(), cmap='gray')